# 第六章 Gradio 的介绍与前端界面的搭建 💬

Gradio 是一种快速便捷的方法，可以直接在 **Python 中通过友好的 Web 界面演示机器学习模型**。在本课程中，我们将学习*如何使用它为生成式人工智能应用程序构建用户界面*。在构建了应用程序的机器学习或生成式人工智能后，如果你想构建一个demo给其他人看，也许是为了获得反馈并推动系统的改进，或者只是因为你觉得这个系统很酷，所以想演示一下：Gradio 可以让您通过 Python 接口程序快速实现这一目标，而无需编写任何前端、网页或 JavaScript 代码。
加载 HF API 密钥和相关 Python 库

In [1]:
os.environ['ZHIPUAI_API_KEY'] ='99ee3e4eb4477848c7a44c0e154a9018.3a7bL4l15TkF3wPi'

In [2]:
# 导入必要的库
import os                # 用于操作系统相关的操作，例如读取环境变量
import io                # 用于处理流式数据（例如文件流）
import IPython.display   # 用于在IPython环境中显示数据，例如图片
import requests          # 用于进行HTTP请求，例如GET和POST请求
import zhipuai
from zhipuai_llm import ZhipuAILLM

# 设置请求的默认超时时间为60秒
requests.adapters.DEFAULT_TIMEOUT = 60

# 导入dotenv库的函数
# dotenv允许您从.env文件中读取环境变量
# 这在开发时特别有用，可以避免将敏感信息（如API密钥）硬编码到代码中
from dotenv import load_dotenv, find_dotenv

# 寻找.env文件并加载它的内容
# 这允许您使用os.environ来读取在.env文件中设置的环境变量
_ = load_dotenv(find_dotenv())

# 从环境变量中读取'ZHIPUAI_API_KEY'并将其存储在hf_api_key变量中
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

我们在这里设置token和辅助函数。你可以看到，在这里我们使用了不同的库。我们使用的是文本生成库，这是一个用于处理开源 LLM 的精简库，可以让你同时加载 API（就像我们在这里做的一样），也可以在本地运行你自己的 LLM。

In [3]:
# 助手函数
llm = ZhipuAILLM(model="chatglm_std", temperature=0)

## 建立一个应用程序，与任何LLM聊天！

在第 2 课中，我们使用了一个非常简单的 Gradio 界面，它有一个文本框输入和一个输出。在这里，我们也可以用类似的方式与 LLM 聊天。再次复制我们的prompt。在这里，我们可以决定需要多少token。这就是非常简单地向 LLM 提问的方法。但我们还是不能聊天，因为如果你再问一个后续问题，它就无法理解或保留上下文。

In [4]:
from __future__ import annotations

import logging
from typing import (
    Any,
    AsyncIterator,
    Dict,
    Iterator,
    List,
    Optional,
)

from langchain.callbacks.manager import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain.llms.base import LLM
from langchain.pydantic_v1 import Field, root_validator
from langchain.schema.output import GenerationChunk
from langchain.utils import get_from_dict_or_env

logger = logging.getLogger(__name__)


class ZhipuAILLM(LLM):
    """Zhipuai hosted open source or customized models.

    To use, you should have the ``zhipuai`` python package installed, and
    the environment variable ``qianfan_ak`` and ``qianfan_sk`` set with
    your API key and Secret Key.

    ak, sk are required parameters which you could get from
    https://cloud.baidu.com/product/wenxinworkshop

    Example:
        .. code-block:: python

            from langchain.llms import QianfanLLMEndpoint
            qianfan_model = QianfanLLMEndpoint(model="ERNIE-Bot",
                endpoint="your_endpoint", ak="your_ak", sk="your_sk")
    """

    model_kwargs: Dict[str, Any] = Field(default_factory=dict)

    client: Any

    model: str = "chatglm_std"
    """Model name in chatglm_pro, chatglm_std, chatglm_lite. """

    zhipuai_api_key: Optional[str] = None

    incremental: Optional[bool] = True
    """Whether to incremental the results or not."""

    streaming: Optional[bool] = False
    """Whether to streaming the results or not."""
    # streaming = -incremental

    request_timeout: Optional[int] = 60
    """request timeout for chat http requests"""

    top_p: Optional[float] = 0.8
    temperature: Optional[float] = 0.95
    request_id: Optional[float] = None

    @root_validator()
    def validate_enviroment(cls, values: Dict) -> Dict:
        values["zhipuai_api_key"] = get_from_dict_or_env(
            values,
            "zhipuai_api_key",
            "ZHIPUAI_API_KEY",
        )

        params = {
            "api_key": values["zhipuai_api_key"],
            "model": values["model"],
        }

        try:
            import zhipuai

            values["client"] = zhipuai.model_api
        except ImportError:
            raise ValueError(
                "zhipuai package not found, please install it with "
                "`pip install zhipuai`"
            )
        return values

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {
            **{"model": self.model},
            **super()._identifying_params,
        }

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "zhipuai"

    @property
    def _default_params(self) -> Dict[str, Any]:
        """Get the default parameters for calling OpenAI API."""
        normal_params = {
            "streaming" :self.streaming,
            "top_p": self.top_p,
            "temperature": self.temperature,
            "request_id": self.request_id,
        }

        return {**normal_params, **self.model_kwargs}

    def _convert_prompt_msg_params(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> dict:
        return {
            **{"prompt": prompt, "model": self.model},
            **self._default_params,
            **kwargs,
        }

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call out to an zhipuai models endpoint for each generation with a prompt.
        Args:
            prompt: The prompt to pass into the model.
        Returns:
            The string generated by the model.

        Example:
            .. code-block:: python
                response = zhipuai_model("Tell me a joke.")
        """
        if self.streaming:
            completion = ""
            for chunk in self._stream(prompt, stop, run_manager, **kwargs):
                completion += chunk.text
            return completion
        params = self._convert_prompt_msg_params(prompt, **kwargs)
        response_payload = self.client.invoke(**params)

        print(response_payload)
        return response_payload["data"]["choices"][-1]["content"]

    async def _acall(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if self.streaming:
            completion = ""
            async for chunk in self._astream(prompt, stop, run_manager, **kwargs):
                completion += chunk.text
            return completion

        params = self._convert_prompt_msg_params(prompt, **kwargs)
        response = await self.client.async_invoke(**params)

        return response_payload

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        params = self._convert_prompt_msg_params(prompt, **kwargs)

        for res in self.client.invoke(**params):
            if res:
                chunk = GenerationChunk(text=res)
                yield chunk
                if run_manager:
                    run_manager.on_llm_new_token(chunk.text)

    async def _astream(

        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> AsyncIterator[GenerationChunk]:
        params = self._convert_prompt_msg_params(prompt, **kwargs)

        async for res in await self.client.ado(**params):
            if res:
                chunk = GenerationChunk(text=res["data"]["choices"]["content"])

                yield chunk
                if run_manager:
                    await run_manager.on_llm_new_token(chunk.text)

In [5]:
llm = ZhipuAILLM(model="chatglm_std")
llm.predict("nih")

{'code': 200, 'msg': '操作成功', 'data': {'request_id': '7982261728276681250', 'task_id': '7982261728276681250', 'task_status': 'SUCCESS', 'choices': [{'role': 'assistant', 'content': '" NIH（National Institutes of Health，美国国立卫生研究院）是美国政府旗下的一个机构，主要负责生物医学研究和公共卫生领域的支持和管理。它成立于 1870 年，总部位于美国马里兰州贝塞斯达市。\\n\\nNIH 主要由 27 个研究所和中心组成，其中包括美国国家癌症研究所、美国国家心血管病研究所、美国国立糖尿病、消化和肾脏病研究所等。它主要负责资助生物医学研究，促进公共健康和医疗水平的提高。同时，NIH 也负责监管和指导生物医学研究的行为和伦理，确保研究的可靠性和公正性。\\n\\n在 2023 年，NIH 的预算约为 47 亿美元，是全球最大的生物医学研究资金提供者之一。它的研究成果广泛应用于临床实践和公共健康领域，对全球医疗健康水平的提高做出了重要贡献。"'}], 'usage': {'prompt_tokens': 137, 'completion_tokens': 177, 'total_tokens': 314}}, 'success': True}


'" NIH（National Institutes of Health，美国国立卫生研究院）是美国政府旗下的一个机构，主要负责生物医学研究和公共卫生领域的支持和管理。它成立于 1870 年，总部位于美国马里兰州贝塞斯达市。\\n\\nNIH 主要由 27 个研究所和中心组成，其中包括美国国家癌症研究所、美国国家心血管病研究所、美国国立糖尿病、消化和肾脏病研究所等。它主要负责资助生物医学研究，促进公共健康和医疗水平的提高。同时，NIH 也负责监管和指导生物医学研究的行为和伦理，确保研究的可靠性和公正性。\\n\\n在 2023 年，NIH 的预算约为 47 亿美元，是全球最大的生物医学研究资金提供者之一。它的研究成果广泛应用于临床实践和公共健康领域，对全球医疗健康水平的提高做出了重要贡献。"'

### 2.1 参数解析

- fn=generate: 这是用于处理输入的函数，即文本生成函数 generate。
- inputs=[
        gr.Textbox(label="Prompt"),
        gr.Slider(label="Temperature", value=0,  maximum=1, minimum=0)。
    ]: 这定义了模型的输入。
    使用 gr.Textbox 部件来以文本框的形式显示输入的内容描述，label 参数设置了输入部件的标签为 prompt。
    使用 gr.Slider 部件以滑动条的形式来显示输入的内容描述，label 参数设置了输入部件的标签为 temperature。
- outputs=[gr.Textbox(label="Caption")]: 这定义了输出部分。使用 gr.Textbox 部件来显示生成的内容描述，label 参数设置了输出部件的标签。
- title="Chat Robot": 这是界面的标题，将显示在界面的顶部。
- description="Local Knowledge Base Q&A with llm ": 这是界面的描述，提供有关界面功能的更多信息。
- allow_flagging="never": 这设置了不允许标记内容，确保不会显示标记不恰当内容的选项。

In [6]:
# 导入所需的库
import gradio as gr  # 用于创建Web界面
import os  # 用于与操作系统交互，如读取环境变量

# 定义一个函数来根据输入生成文本
def generate(input, slider):
    # 使用预定义的client对象的generate方法，从输入生成文本
    # slider的值限制生成的token的数量
    output = llm.predict(input, temperature=slider)
    return output  # 返回生成的文本

# 创建一个Web界面
# 输入：一个文本框和一个滑块
# 输出：一个文本框显示生成的文本
demo = gr.Interface(
    fn=generate, 
    inputs=[
        gr.Textbox(label="Prompt"),  # 文本输入框
        gr.Slider(label="Temperature", value=0,  maximum=1, minimum=0)  # 滑块用于选择模型的 temperature
    ], 
    outputs=[gr.Textbox(label="Completion")],  # 显示生成文本的文本框
    title="Chat Robot",  # 界面标题
    description="Local Knowledge Base Q&A with llm",  # 界面描述
    # allow_flagging="never", 
)

# 关闭可能已经启动的任何先前的gradio实例
gr.close_all()

# 启动Web界面
# 使用环境变量PORT1作为服务器的端口号
# demo.launch(share=True, server_port=int(os.environ['PORT1']))
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


现在我们已经搭建了一个非常简单的 Gradio 界面，它有一个文本框输入和一个输出。我们已经可以非常简单地向 LLM 提问。但我们还是不能聊天，因为如果你再问一个后续问题，它就无法理解或保留上下文。

因此，基本上我们要做的是，向模型发送我们之前的问题、它自己的回答以及后续问题。但建立所有这些都有点麻烦。这就是 Gradio 聊天机器人组件的作用所在，因为它允许我们简化向模型发送对话历史记录的过程。

因此，我们要解决这个问题。为此，我们将引入一个新的 Gradio 组件--Gradio Chatbot。

![math](images/ch06_math.png)

## 使用 `gr.Chatbot()` 来助力!

让我们开始使用 Gradio Chatbot 组件。这里实例化了一个带有文本框提示和提交按钮的 Gradle ChatBot 组件，是一个非常简单的用户界面。但我们现在还不是在和 LLM 聊天。

只需随机选择三个预设回复，然后将我的信息和机器人的信息添加到聊天记录中。所以在这里，你可以看到我可以说任何话，它基本上会随机查看这三个回复。

![math_with_template](images/ch06_math_with_template.png)

我们必须格式化聊天prompt。此处正在定义这个格式化聊天prompt函数。
在这里，我们要做的就是使其包含聊天历史记录，这样 LLM 就能知道上下文。
但这还不够。我们还需要告诉它，哪些信息来自用户，哪些信息来自 LLM 本身，也就是我们正在调用的助手。
因此，我们设置了格式聊天prompt功能，在聊天记录的每一轮中，都包含一条用户信息和一条助手信息，以便我们的模型能准确回答后续问题。
现在，我们要将格式化的prompt传递给我们的 API。

In [10]:
client = llm

In [11]:
# 定义一个函数，用于格式化聊天提示。
def format_chat_prompt(message, chat_history):
    # 初始化一个空字符串，用于存放格式化后的聊天提示。
    prompt = ""
    # 遍历聊天历史记录。
    for turn in chat_history:
        # 从聊天记录中提取用户和机器人的消息。
        user_message, bot_message = turn
        # 更新提示，加入用户和机器人的消息。
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    # 将当前的用户消息也加入到提示中，并预留一个位置给机器人的回复。
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    # 返回格式化后的提示。
    return prompt

# 定义一个函数，用于生成机器人的回复。
def respond(message, chat_history):
    # 调用上面的函数，将用户的消息和聊天历史记录格式化为一个提示。
    formatted_prompt = format_chat_prompt(message, chat_history)
    # 使用client对象的generate方法生成机器人的回复（注意：client对象在此代码中并未定义）。
    bot_message = client.predict(formatted_prompt,
                                  max_new_tokens=1024,
                                  stop_sequences=["\nUser:", ""])
    # 将用户的消息和机器人的回复加入到聊天历史记录中。
    chat_history.append((message, bot_message))
    # 返回一个空字符串和更新后的聊天历史记录（这里的空字符串可以替换为真正的机器人回复，如果需要显示在界面上）。
    return "", chat_history

# 下面的代码是设置Gradio界面的部分。

# 使用Gradio的Blocks功能定义一个代码块。
with gr.Blocks() as demo:
    # 创建一个Gradio聊天机器人组件，设置其高度为240。
    chatbot = gr.Chatbot(height=240) 
    # 创建一个文本框组件，用于输入提示。
    msg = gr.Textbox(label="Prompt")
    # 创建一个提交按钮。
    btn = gr.Button("Submit")
    # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # 设置按钮的点击事件。当点击时，调用上面定义的respond函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 

# 关闭所有已经存在的Gradio实例。
gr.close_all()
# 启动新的Gradio应用，设置分享功能为True，并使用环境变量PORT3指定服务器端口。
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


![animal](images/ch06_animal.png)

![animal_in_context](images/ch06_animal_in_context.png)

现在，我们的聊天机器人应该可以回答后续问题了。
我们可以看到，我们向它发送了上下文。我们向它发送了信息，然后要求它完成。一旦我们进入另一个迭代循环，我们就会向它发送我们的整个上下文，然后要求它完成。这很酷。但是，如果我们一直这样迭代下去，那么模型在一次对话中所能接受的信息量就会达到极限，因为我们总是给它越来越多的之前对话的内容。

为了让模型发挥最大作用，我们可以在这里将最大token数`max_new_tokens`设置为 1024、 这是我们在 API 中运行的硬件条件下，该模型所能接受的最大值。

可以尝试以下prompt：
1. 哪些动物生活在热带草原？
2. 这之中哪种动物最强壮？

这里，我们创建了一个简单但功能强大的用户界面，用于与LLM聊天。如果需要进一步Gradio 所能提供的最佳功能，我们可以创建一个包含更多功能的用户界面。

### 添加其他高级功能

In [12]:
# 定义一个函数，用于格式化聊天提示。
def format_chat_prompt(message, chat_history, instruction):
    # 初始化提示，加入系统指令。
    prompt = f"System:{instruction}"
    # 遍历聊天历史记录。
    for turn in chat_history:
        # 从聊天记录中提取用户和机器人的消息。
        user_message, bot_message = turn
        # 更新提示，加入用户和机器人的消息。
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    # 将当前的用户消息也加入到提示中，并预留一个位置给机器人的回复。
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    # 返回格式化后的提示。
    return prompt

# 定义一个函数，用于生成机器人的回复。
def respond(message, chat_history, instruction, temperature=0.7):
    # 调用上面的函数，将用户的消息、聊天历史记录和系统指令格式化为一个提示。
    prompt = format_chat_prompt(message, chat_history, instruction)
    # 更新聊天历史记录，先加入用户的消息（机器人的回复部分先为空）。
    chat_history = chat_history + [[message, ""]]
    # 使用client对象的generate_stream方法生成机器人的回复（注意：client对象在此代码中并未定义）。
    stream = client.generate_stream(prompt,
                                    max_new_tokens=1024,
                                    stop_sequences=["\nUser:", ""], 
                                    temperature=temperature)  # 设置生成回复的温度，决定回复的随机性。
    acc_text = ""
    # 使用流式处理获取机器人的回复。
    for idx, response in enumerate(stream):
        text_token = response.token.text

        # 如果有任何详情信息，直接返回。
        if response.details:
            return

        # 如果是第一个令牌并且它以空格开始，则去除该空格。
        if idx == 0 and text_token.startswith(" "):
            text_token = text_token[1:]

        # 累积生成的文本。
        acc_text += text_token
        # 更新最后一轮的聊天记录。
        last_turn = list(chat_history.pop(-1))
        last_turn[-1] += acc_text
        chat_history = chat_history + [last_turn]
        yield "", chat_history
        acc_text = ""

# 设置Gradio界面部分。
with gr.Blocks() as demo:
    # 创建一个Gradio聊天机器人组件，并设置其高度。
    chatbot = gr.Chatbot(height=240)
    # 创建一个文本框组件，用于输入提示。
    msg = gr.Textbox(label="Prompt")
    # 创建一个可折叠组件，用于显示高级选项。
    with gr.Accordion(label="Advanced options", open=False):
        # 在可折叠组件内创建一个文本框，用于输入系统消息。
        system = gr.Textbox(label="System message", lines=2, value="一段用户和基于大语言模型的法律助手的对话. 助手会给出真实且有帮助的回答.")
        # 创建一个滑块，用于调整回复的温度。
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    # 创建一个提交按钮。
    btn = gr.Button("Submit")
    # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # 设置按钮的点击事件。当点击时，调用上面定义的respond函数，并传入用户的消息、聊天历史记录和系统消息，然后更新文本框和聊天机器人组件。
    btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
    # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
    msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])

![law_1](images/ch06_law_1.png)

![law_2](images/ch06_law_2.png)

![law_3](images/ch06_law_3.png)

在这里，我们有高级选项，包括系统消息，它可以设置 LLM 与你聊天的模式。
因此，在系统消息中，你可以说，例如，你是一个乐于助人的助手，或者你可以给它一个特定的语气，一个特定的语调，
你希望它更有趣一点，更严肃一点，你真的可以反复调试系统消息提示，看看它对你的消息有什么影响。

有些人甚至会想给 LLM 一个角色，比如你是一个提供法律建议的律师，或者你是一个提供医疗建议的医生，
但要注意的是，众所周知，LLM 会以一种听起来很真实的方式提供虚假信息。
因此，尽管使用Falcon 40B 进行实验和探索会很有趣，但在现实世界的应用场景中，必须为此类使用案例制定进一步的保障措施。

还有其他高级参数，比如这里的温度。
温度基本上就是你希望模型的变化程度。因此，如果将温度设为零，模型就会倾向于始终对相同的输入做出相同的反应。
所以同样的问题，同样的答案。温度越高，信息的变化就越多。但如果温度过高，它就会开始给出无意义的答案。
因此，0.7 是一个不错的默认参数，但我们鼓励你多做尝试。

除此之外，这个用户界面还能让我们进行流式传输回复。
它逐个token发送，我们可以看到它实时完成。因此，我们不需要等到整个答案都准备好了。在这里，我们可以看到它是如何完成的。如果你不理解这里的所有内容，也不用担心，因为我们的目的是用一个非常完整的用户界面来结束课程，并提供LLM方面的所有功能。

在格式聊天提示中，也就是我们之前使用的功能中，我们添加了一个新元素，那就是系统指令。因此，在开始用户助手对话之前，我们在系统顶部添加了一个指令。因此，基本上在发送给模型的每条信息的开头，都会有我们设置的系统信息。在这里，我们调用文本生成库的`generate_stream`函数。而 `generate_stream`函数的作用就是逐个生成响应标记。因此，在这个循环中，发生的事情就是按标记生成响应标记，将其添加到聊天记录中，然后将其返回给函数。

In [13]:
# 关闭可能已经启动的任何先前的gradio实例
gr.close_all()

Closing server running on port: 7860


现在我们可以将本地数据库的内容接入进来，让 llm 通过本地数据库进行回答。

In [14]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

In [15]:
from langchain.prompts import PromptTemplate

In [34]:
persist_directory = '../docs/chroma/knowledge_base'

In [35]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
def chat_with_db(query, chat_history):

    llm = ZhipuAILLM(model="chatglm_std")

    embedding = ZhipuAIEmbeddings()
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    # Build prompt
    template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
    {context}
    问题：{question}
    有用的回答："""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    # qa_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )
    result = qa_chain({"query": query, "context": chat_history})
    
    # 将用户的消息和机器人的回复加入到聊天历史记录中。
    chat_history.append((query, result['result']))
    # 返回一个空字符串和更新后的聊天历史记录（这里的空字符串可以替换为真正的机器人回复，如果需要显示在界面上）。
    return "", chat_history

    # return result['answer'], chat_history

In [33]:
!pwd

/Users/zhihu123/Project/other/Tutorial_for_developing_LLM_application/notebook/C7  前后端搭建


In [36]:
# 使用Gradio的Blocks功能定义一个代码块。
with gr.Blocks() as demo:
    # 创建一个Gradio聊天机器人组件，设置其高度为240。
    chatbot = gr.Chatbot(height=240) 
    # 创建一个文本框组件，用于输入提示。
    msg = gr.Textbox(label="Prompt")
    # 创建一个提交按钮。
    btn = gr.Button("Submit")
    # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # 设置按钮的点击事件。当点击时，调用上面定义的 chat_with_db 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
    btn.click(chat_with_db, inputs=[msg, chatbot], outputs=[msg, chatbot])
    # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
    msg.submit(chat_with_db, inputs=[msg, chatbot], outputs=[msg, chatbot]) 

# 关闭所有已经存在的Gradio实例。
gr.close_all()
# 启动新的Gradio应用，设置分享功能为True，并使用环境变量PORT3指定服务器端口。
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


向量库中存储的数量：0
{'query': '课程', 'context': []}
['query']
{'input_documents': [], 'question': '课程'}
['input_documents', 'question']
{'question': '课程', 'context': ''}
['context', 'question']
